In [7]:
import h3

# Write data into Snowflake table
import snowflake.connector
import pandas as pd

#----------CONNECTION TO SNOWFLAKE-------------
SF_ACCOUNT = 'gfa04036.us-east-1'
SF_WH = 'TRANSFORMING'
SF_USERNAME = 'DBT_USER'
SF_PASSWORD = '2C>`8Q!8y*Sz]h/):Xxy&WNJv'

# Connecting to Snowflake using the default authenticator
ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()

In [8]:
# # Reading data
# cur=ctx.cursor()

# # Lets aggregate the metering data for Dominion Power up to the day level
# #sql = "select date_trunc('day', STARTTIME), count(*) from trips_vw" +\
# #        " group by date_trunc('day', STARTTIME)" +\
# #        " order by date_trunc('day', STARTTIME)"
# sql =   """
#         SELECT
#             *
#         FROM raw.co_meiko.cali
#         """

# cur.execute(sql)

# # Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
# clients = cur.fetch_pandas_all()
# clients.reset_index(inplace=True)
# clients = clients.to_dict(orient = 'index')

clients = pd.read_csv('Meiko Total COL.csv')
clients.reset_index(inplace=True)
clients = clients.to_dict(orient='index')

# For each client, try to write the cell
for i in clients:
    clients[i]['r12_h3'] = h3.geo_to_h3(clients[i]['LATITUD'], clients[i]['LONGITUDE'], 12)
    clients[i]['r11_h3'] = h3.geo_to_h3(clients[i]['LATITUD'], clients[i]['LONGITUDE'], 11)
    clients[i]['ID_MEIKO'] = clients[i]['idEstablecimientoMeiko']

print(len(clients))

13065


In [9]:
ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()
sql = """ CREATE OR REPLACE TABLE stg_py_meiko_h3_cells (
                source_country varchar(1020),
                meiko_id number(38,0),
                r11_h3 varchar(1020),
                r12_h3 varchar(1020)) """

cur.execute(sql)
#DataFrame of the needed results to update in the same orde
# multiple records to be updated in tuple format
sql = "INSERT INTO stg_py_meiko_h3_cells VALUES"
count = 0
for i in clients:
    count += 1
    sql += "(" \
            + "'" + "COLOMBIA'" + "," \
            + "'" + str(clients[i]['ID_MEIKO']) + "'" + "," \
            + "'" + clients[i]['r11_h3'] + "'" + ","  \
            + "'" + clients[i]['r12_h3'] + "'" \
            + ")"
    if (count % 10000 == 0):
                cur.execute(sql)
                print("Flushing 10000 records: ", count)
                sql = "INSERT INTO stg_py_meiko_h3_cells VALUES "
    else:
        if count < len(clients):
            sql += ","

cur.execute(sql)    
print("Records updated successfully", count)
cur.close()

Flushing 10000 records:  10000
Records updated successfully 13065


True